In [42]:
from utils.tf import *
from utils.db import *
from utils.gd2 import *
from utils.clt_repetidos import *
from datetime import datetime, time, timedelta
import cx_Oracle
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go
from utils.clt_repetidos import * 
from ydata_profiling import ProfileReport
import dask.dataframe as dd
import gc
import matplotlib.pyplot as plt
from datetime import datetime


In [2]:
df_flujo = pd.read_csv("/home/ale1726/proyects/datalake/clientes/data/otros/data_informe_20250508.csv")

In [4]:
df_flujo["Fecha"] = pd.to_datetime(df_flujo["Fecha"])

In [8]:
df_flujo = df_flujo.sort_values(by='Fecha', ascending=False)

In [32]:
df_flujo["SISTEMA"].nunique()


20

In [ ]:
df_filtred = df_flujo[df_flujo["Fecha"] == df_flujo["Fecha"].max()]
len(df_filtred[(df_filtred["Acesso"] == "SI") & ((df_filtred["Acesso_tabla"] == "SI") | (df_filtred["Acesso_tabla"] == "N/C"))])
len(df_filtred[(df_filtred["Acesso"] == "SI") & (df_filtred["Acesso_tabla"] == "NO")])
df_filtred["Accesso_num"].eq(0).sum()

In [43]:
def mapear_a_horario_mas_cercano(fecha):
    fecha_base = fecha.date()
    hora_930 = datetime.combine(fecha_base, time(9, 30))
    hora_1730 = datetime.combine(fecha_base, time(17, 30))
    
    # Calcula la diferencia absoluta en tiempo
    diff_930 = abs(fecha - hora_930)
    diff_1730 = abs(fecha - hora_1730)
    
    # Retorna el más cercano
    return hora_930 if diff_930 < diff_1730 else hora_1730

In [45]:
df_flujo['Fecha_mapeada'] = df_flujo['Fecha'].apply(mapear_a_horario_mas_cercano)

In [50]:
df_sin_duplicados = df_flujo.drop_duplicates(subset=['Fecha_mapeada', 'SISTEMA'])

In [63]:
fecha_max = df_sin_duplicados["Fecha_mapeada"].max()
fecha_inicio = (fecha_max - pd.Timedelta(days=7)).replace(hour=9, minute=30, second=0)
df_filtro_7dias = df_sin_duplicados[(df_sin_duplicados["Fecha"] >= fecha_inicio) & (df_sin_duplicados["Fecha"] <= fecha_max)]

In [64]:
df_filtro_7dias

,SISTEMA,Fecha,Acesso,Acesso_tabla,Tipo_de_error,Accesso_num,Fecha_mapeada
1672,BNC,2025-05-08 17:30:00,SI,N/C,Conexion correcta,1,2025-05-08 17:30:00
1668,SIAF,2025-05-08 17:30:00,ERROR,NO,ORA-28001: the password has expired,0,2025-05-08 17:30:00
1676,SCC,2025-05-08 17:30:00,SI,NO,Conexion a la base de datos pero no a la tabla...,1,2025-05-08 17:30:00
1675,CCBE,2025-05-08 17:30:00,SI,N/C,Conexion correcta,1,2025-05-08 17:30:00
1674,IMRC,2025-05-08 17:30:00,SI,N/C,Conexion correcta,1,2025-05-08 17:30:00
...,...,...,...,...,...,...,...
3745,SOME,2025-05-01 09:30:00,SI,NO,Conexion a la base de datos pero no a la tabla...,1,2025-05-01 09:30:00
3744,SOI,2025-05-01 09:30:00,ERROR,NO,ORA-01017: invalid username/password; logon de...,0,2025-05-01 09:30:00
3743,SIPE,2025-05-01 09:30:00,SI,SI,Conexion correcta,1,2025-05-01 09:30:00
3749,SORI,2025-05-01 09:30:00,SI,SI,Conexion correcta,1,2025-05-01 09:30:00
